In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import usaddress
import re
import json

First thing we need to do is load the dataset. Pandas conveniently understands URLs, so we can load them directly from the website.

In [2]:
# If you're running this on your own on a fast connection, this will work. 
# I recommend writing out the dataframe later.
# tax_bills_june15_bbls = pd.read_csv("http://taxbills.nyc/tax_bills_june15_bbls.csv")
# tax_bills_june15_exab = pd.read_csv("http://taxbills.nyc/tax_bills_june15_exab.csv")

# loading these over HTTP turns out to be an arduous task
tax_bills_june15_bbls = pd.read_csv("/Users/jeff/Downloads/tax_bills_june15_bbls.csv", index_col='bbl')
tax_bills_june15_exab = pd.read_csv("/Users/jeff/Downloads/tax_bills_june15_exab.csv", index_col='bbl')


In [3]:
tax_bills_june15_bbls

,ownername,address,taxclass,taxrate,emv,tbea,bav,tba,propertytax,condonumber,condo
bbl,,,,,,,,,,,
1000010010,GOVERNORS ISLAND CORPORATION,GOVERNORS ISLAND CORPORATION\nC/O TRUST FOR. G...,4 - commercial property,10.6840%,337672000.0,15749050.0,147407802.0,NaN,0.0,NaN,NaN
1000010101,U. S. GOVT LAND & BLDGS,"BEDLOES ISLAND\n1 LIBERTY ISLAND\nELLIS ISLAND,",4 - commercial property,10.6840%,25607000.0,1106496.0,10356570.0,NaN,0.0,NaN,NaN
1000010201,U. S. GOVT LAND & BLDGS,"ELLIS ISLAND\n1 LIBERTY ISLAND\nELLIS ISLAND,",4 - commercial property,10.6840%,233982000.0,10366655.0,97029720.0,NaN,0.0,NaN,NaN
1000020001,NYC DSBS,"NYC DSBS\n110 WILLIAM ST. FL. 7\nNEW YORK , NY...",4 - commercial property,10.6840%,69458000.0,3163690.0,29611473.0,NaN,0.0,NaN,NaN
1000020002,"10 SSA LANDLORD, LLC","10 SSA LANDLORD, LLC\n729 7TH AVE. FL. 15\nNEW...",4 - commercial property,10.6840%,55592000.0,2672762.0,25016491.0,654246.0,654246.0,NaN,NaN
1000020003,NOT ON FILE,"\nBAD LOCATION ADDRESS\n,",4 - commercial property,10.6840%,1774000.0,83277.0,779458.0,83277.0,83277.0,NaN,NaN
1000020023,NYC DSBS,"NYC DSBS\n110 WILLIAM ST. FL. 7\nNEW YORK , NY...",4 - commercial property,10.6840%,36968000.0,1824996.0,17081581.0,NaN,0.0,NaN,NaN
1000030001,PARKS AND RECREATION (GENERAL),PARKS AND RECREATION (GENERAL)\nARSENAL WEST\n...,4 - commercial property,10.6840%,285745000.0,13749587.0,128693250.0,NaN,0.0,NaN,NaN
1000030002,PARKS AND RECREATION (GENERAL),PARKS AND RECREATION (GENERAL)\nARSENAL WEST\n...,4 - commercial property,10.6840%,10918000.0,524916.0,4913100.0,NaN,0.0,NaN,NaN


In [4]:
tax_bills_june15_exab

,type,detail,amount,units
bbl,,,,
4015180009,abatement,j51 abatement,-4075.0,NaN
4046020125,abatement,j51 abatement,-11794.0,NaN
4001570040,abatement,j51 abatement,-6942.0,NaN
4004740010,exemption,icip,-3548.0,NaN
4012820175,abatement,j51 abatement,-8735.0,NaN
4096480024,abatement,j51 abatement,-9421.0,NaN
4008811001,exemption,icip,-9702.0,NaN
4066880010,exemption,clergy,-287.0,NaN
4022201059,abatement,j51 abatement,-69.0,NaN


Note that address isn't exactly right. We'll have to normalize it a bit before we go further.  First thing to do is get rid of the pesky newlines.  It turns out they're not really newlines, since we read a CSV, but rather literal escapes. 

My first thought was to simply get rid of them as in below:

In [5]:
corrected_addresses = tax_bills_june15_bbls['address'].str.replace("\\\\n", ' ')
next(iter(corrected_addresses))

'GOVERNORS ISLAND CORPORATION C/O TRUST FOR. GOVERNORS ISLAN 10 SOUTH ST. APT. FRNT SLIP7'

But it turns out that a lot of geocoders don't respond well to recipient names at the head of the address, zip4s, and borough names instead of city names. Our solution won't be perfect, but it'll be decent enough to show as an example and leave "perfection" for the reader if it's really necessary for the application.  

Nominatim in particular is fragile, but we use it because it's free and requires no API key.  We also use it because in order to do bulk geocoding you're going to have to set up Nominatim yourself or another equally fragile bulk geocoder (there are several).

So let's go ahead and do the corrections.

First we'll correct some common quirks. We'll drop empty addresses and replace "One" with 1, which is common in commercial districts.

In [49]:
# starts_with_number = re.compile('^[0-9]+')
starts_with_one = re.compile('^ONE')

def scrub_addr(addr_lines):
    global starts_with_number
    if not isinstance(addr_lines, list):
        return addr_lines
    
    if starts_with_one.match(addr_lines[0]):  # this is a common pattern in addresses
        addr_lines[0] = addr_lines[0].replace('ONE', '1')
        
    if len(addr_lines):
        return ' '.join(addr_lines)
    else:
        return np.nan

This next function will take the addresses we ahve and make sure they at least have state and zipcode attached.

In [ ]:
state_and_zip = re.compile(r'^.*NY\s+(?:[0-9]{5})?(?:-[0-9]{4})\s*(?:USA)?$')

def append_state_info(addr):
    if not state_and_zip.match(addr):
        return addr + ', NY 00000'  # we append a dummy zipcode because it helps the address tagger work better.
    else:
        return addr

Now that we've cleaned up common quirks, let's tag the address. Rule number 1. Addresses, despite being ubiquitous, are _messy_. **There is an (likely open-source) address parser for your country. Find it and use it. Don't try to create your own**. Writing your own address parser will cause damage to your monitor, keyboard, desk, and face.  We don't want that, do we? 

My code is using `usaddress`.  It's slow but accurate, so we'll not actually be doing the entire dataset in this notebook.  You can of course do the whole dataset yourself without trouble, but you should really go make some pancakes from scratch and eat them while you're waiting for it to finish.

In [ ]:
def tag_addr(addr):
    if not isinstance(addr, str) or len(addr) == 0:
        return np.nan
    else:
        try:
            return usaddress.tag(addr)
        except usaddress.RepeatedLabelError:  # some addresses turn out to have bits repeated
            return tag_addr(' '.join(addr.split(' ')[1:]))

A final pass will re-merge the address into a single string for geocoding, keeping only the parts of the address that the geocoder understands. 

In [56]:
def join_addr(addr):
    addr = addr[0]
    if 'ZipCode' in addr and '-' in addr['ZipCode']:
        addr['ZipCode'] = addr['ZipCode'].split('-')[0]   # Nominatim hates zip4
        
    if 'PlaceName' not in addr:
        addr['PlaceName'] = 'New York'  # we already know we're in new york, some addresses omit it.
    
    if all((
        'AddressNumber' in addr,
        'StreetName' in addr,
        'ZipCode' in addr and addr['ZipCode'] != '00000'
    )):
        return ' '.join((
                addr.get('AddressNumberPrefix', ''),
                addr.get('AddressNumber', ''),
                addr.get('AddressNumberSuffix', ''),
                addr.get('StreetNamePreModifier', ''),
                addr.get('StreetNamePreDirectional', ''),
                addr.get('StreetNamePreType', ''),
                addr.get('StreetName', ''),
                addr.get('StreetNamePostType', ''),
                addr.get('StreetNamePostDirectional', ''),
                addr['StateName'], 
                addr['ZipCode']
            ))
    elif all((
        'AddressNumber' in addr,
        'StreetName' in addr
    )):
        return ' '.join((
            addr.get('AddressNumberPrefix', ''),
            addr.get('AddressNumber', ''),
            addr.get('AddressNumberSuffix', ''),
            addr.get('StreetNamePreModifier', ''),
            addr.get('StreetNamePreDirectional', ''),
            addr.get('StreetNamePreType', ''),
            addr.get('StreetName', ''),
            addr.get('StreetNamePostType', ''),
            addr.get('StreetNamePostDirectional', ''),
            addr['StateName'],     
        ))
    else:
        return np.nan

In [57]:
    
scrubbed_addresses = tax_bills_june15_bbls['address']\
    .sample(n=150)\
    .str.split("\\\\n")\
    .map(scrub_addr)\
    .dropna()\
    .map(append_state_info)\
    .map(tag_addr)\
    .map(join_addr)\
    .dropna()

len(scrubbed_addresses)

104

In [59]:
geocodable_tax_bills_june15_bbls = tax_bills_june15_bbls.ix[scrubbed_addresses.index]
geocodable_tax_bills_june15_bbls['address'] = scrubbed_addresses
geocodable_tax_bills_june15_bbls

,ownername,address,taxclass,taxrate,emv,tbea,bav,tba,propertytax,condonumber,condo
bbl,,,,,,,,,,,
4050221346,"WINSTON TOWER, LLC",315 CENTRAL PARK W. NY,"2 - residential, more than 10 units",12.8550%,47791.0,2532.0,19698.0,2532.0,2532.0,162.0,unit
5022680027,ALAN NANCY EILENBERG,228 LONDON RD. NY 10306,"1 - small home, less than 4 families",19.1570%,714000.0,8066.0,42103.0,7764.0,7764.0,NaN,NaN
4011690028,"TZIKAS, NICK (TRUSTEE)",3242 74TH ST. NY 11370,"1 - small home, less than 4 families",19.1570%,591000.0,5902.0,30811.0,5600.0,5600.0,NaN,NaN
1000151174,"NEKKAH, KEVIN ERWIN",20 WEST ST. NY,"2 - residential, more than 10 units",12.8550%,165429.0,8485.0,66004.0,3072.0,2074.0,1557.0,unit
4001391039,"AMINOV, BARNO",16427 75TH RD. NY 11366,"2 - residential, more than 10 units",12.8550%,43144.0,2349.0,18274.0,2349.0,2349.0,646.0,unit
3053400039,"CHAMBERS, DESMOND",246 E. 8TH ST. NY 11218,"1 - small home, less than 4 families",19.1570%,877000.0,5058.0,26403.0,4756.0,4756.0,NaN,NaN
4072510031,"WOISLAVSKY, IRWIN",18442 TUDOR RD. NY 11432,"1 - small home, less than 4 families",19.1570%,976000.0,9572.0,49968.0,9270.0,9270.0,NaN,NaN
4132000004,"PATEL, JYOTI",13560 234TH PL. NY 11422,"1 - small home, less than 4 families",19.1570%,446000.0,4298.0,22438.0,4298.0,4298.0,NaN,NaN
2038730049,"GUERRA, LOUIS J.",1345 ROSEDALE AVE. NY 10472,2a - 4-6 unit residential building,12.8550%,448000.0,6261.0,48708.0,5645.0,5645.0,NaN,NaN


Now these look good.  However, geocoding all of them with `geopy` will prove impossible. Not only will it take until sometime next year to complete because of rate limiting, but you will find that geocoding services like to charge a fee for their services beyond a certain number of records. In awhile we'll show you how to load your own geocoder and use it.

For now, we'll take a subset.  Some services rate limit you to one call a second.  To avoid taking forever, we'll just use a few records at first.

In [52]:
# grab a random sample of records
tax_bills_bbls_sample = geocodable_tax_bills_june15_bbls.sample(n=15)

# grab the same records from the key-linked dataframe.
tax_bills_exab_sample = tax_bills_june15_exab.ix[tax_bills_bbls_sample.index]

In [53]:
tax_bills_bbls_sample

,ownername,address,taxclass,taxrate,emv,tbea,bav,tba,propertytax,condonumber,condo
bbl,,,,,,,,,,,
4006480018,"RABOS, CONSTANTINE",3157 35TH ST. NY 11106,2a - 4-6 unit residential building,12.8550%,818000.0,12720.0,98946.0,12720.0,12720.0,NaN,NaN
4134860062,"WILSON, MARJORIE",14536 230TH ST. NY 11413,"1 - small home, less than 4 families",19.1570%,494000.0,5020.0,26203.0,4718.0,4718.0,NaN,NaN
4081360040,DENNIS DELORENZO,4132 WESTMORELAND ST. NY 11363,"1 - small home, less than 4 families",19.1570%,1350000.0,9973.0,52058.0,9671.0,9671.0,NaN,NaN
3042680035,JAMES OXLEY FAMILY IRREVOCABLE TRUST,664 HEMLOCK ST. NY 11208,"1 - small home, less than 4 families",19.1570%,440000.0,4735.0,24715.0,4433.0,4433.0,NaN,NaN
3053990052,MELVIN BRICKMAN,507 F. NY 11218,"1 - small home, less than 4 families",19.1570%,929000.0,7885.0,41158.0,7583.0,7583.0,NaN,NaN
3001070024,PARKS AND RECREATION (GENERAL),16 61ST ST. New York NY,4 - commercial property,10.6840%,182000.0,8471.0,79290.0,NaN,0.0,NaN,NaN
3073330067,FRANKIE KAFAI LAU,2053 E. 28TH ST. NY 11229,"1 - small home, less than 4 families",19.1570%,408000.0,4690.0,24480.0,4388.0,4388.0,NaN,NaN
4104270034,"PATRICK, GLORIA",18833 KEESEVILLE AVE. NY 11412,"1 - small home, less than 4 families",19.1570%,450000.0,3761.0,19634.0,3459.0,3459.0,NaN,NaN
1006440063,"FAIRFAX & SAMMONS PROPERTIES, LLC",67 GANSEVOORT ST. NY 10014,4 - commercial property,10.6840%,4157000.0,157150.0,1470890.0,157150.0,157150.0,NaN,NaN


Now a quick sanity check to make sure our addresses work.

In [60]:
print(next(iter(tax_bills_bbls_sample['address'])))
geocoder = Nominatim()
geocoder.geocode(next(iter(tax_bills_bbls_sample['address'])))


3157  35TH ST. NY 11106


Location(35th Street, Astoria, Queens County, NYC, New York, 11101, United States of America, (40.7628959, -73.9202263, 0.0))

In [61]:
geocoded_addresses = tax_bills_bbls_sample['address'].map(lambda addr: geocoder.geocode(addr))
geocoded_addresses

bbl
4006480018    (35th Street, Astoria, Queens County, NYC, New...
4134860062    (230th Street, Laurelton, Queens County, NYC, ...
4081360040    (Westmoreland Street, Douglaston, Queens Count...
3042680035    (664, Hemlock Street, East New York, Kings Cou...
3053990052    (507, Avenue F, Parkville, BK, Kings County, N...
3001070024    (60-16, 61st Street, Fresh Pond, Queens County...
3073330067    (2053, East 28th Street, Sheepshead Bay, BK, K...
4104270034    (Keeseville Avenue, Saint Albans, Queens Count...
1006440063    (67, Gansevoort Street, Chelsea, Manhattan, Ne...
4082470008    (Thebes Avenue, Little Neck, Queens County, NY...
4157090005    (Loretta Road, Roy Reuther Houses, Far Rockawa...
1008701214    (72 1/2, Irving Place, Flatiron, Manhattan, Ne...
3032750036    (96, Himrod Street, Bushwick, Kings County, NY...
2033630067    (4222, Herkimer Place, Woodlawn, Bronx, Bronx ...
4114174401    (149th Avenue, Ozone Park, Kings, NYC, New Yor...
Name: address, dtype: object

Now we have some geocoded addresses! We got lucky here, but in a larger sample, some will be `None`. A `.dropna()` or filter should suffice to get rid of null values. Each location will have a bunch of fields, but the most useful are the canonical address, longitude, and latitude:

In [65]:
loc = next(iter(geocoded_addresses))  # let's inspect the first one.
print((loc.address, (loc.longitude, loc.latitude)))

('35th Street, Astoria, Queens County, NYC, New York, 11101, United States of America', (-73.9202263, 40.7628959))


Now the only thing left is to add Series for these to our sample.  I like to canonicalize the address, too, although you may want to rename the original address field to `original_address` or some such: 

In [66]:
tax_bills_bbls_sample['address'] = geocoded_addresses.map(lambda l: l.address)
tax_bills_bbls_sample['latitude'] = geocoded_addresses.map(lambda l: l.latitude)
tax_bills_bbls_sample['longitude'] = geocoded_addresses.map(lambda l: l.longitude)
tax_bills_bbls_sample

,ownername,address,taxclass,taxrate,emv,tbea,bav,tba,propertytax,condonumber,condo,latitude,longitude
bbl,,,,,,,,,,,,,
4006480018,"RABOS, CONSTANTINE","35th Street, Astoria, Queens County, NYC, New ...",2a - 4-6 unit residential building,12.8550%,818000.0,12720.0,98946.0,12720.0,12720.0,NaN,NaN,40.762896,-73.920226
4134860062,"WILSON, MARJORIE","230th Street, Laurelton, Queens County, NYC, N...","1 - small home, less than 4 families",19.1570%,494000.0,5020.0,26203.0,4718.0,4718.0,NaN,NaN,40.659682,-73.750916
4081360040,DENNIS DELORENZO,"Westmoreland Street, Douglaston, Queens County...","1 - small home, less than 4 families",19.1570%,1350000.0,9973.0,52058.0,9671.0,9671.0,NaN,NaN,40.772826,-73.738065
3042680035,JAMES OXLEY FAMILY IRREVOCABLE TRUST,"664, Hemlock Street, East New York, Kings Coun...","1 - small home, less than 4 families",19.1570%,440000.0,4735.0,24715.0,4433.0,4433.0,NaN,NaN,40.672613,-73.868584
3053990052,MELVIN BRICKMAN,"507, Avenue F, Parkville, BK, Kings County, NY...","1 - small home, less than 4 families",19.1570%,929000.0,7885.0,41158.0,7583.0,7583.0,NaN,NaN,40.633838,-73.973400
3001070024,PARKS AND RECREATION (GENERAL),"60-16, 61st Street, Fresh Pond, Queens County,...",4 - commercial property,10.6840%,182000.0,8471.0,79290.0,NaN,0.0,NaN,NaN,40.714971,-73.902534
3073330067,FRANKIE KAFAI LAU,"2053, East 28th Street, Sheepshead Bay, BK, Ki...","1 - small home, less than 4 families",19.1570%,408000.0,4690.0,24480.0,4388.0,4388.0,NaN,NaN,40.601201,-73.943775
4104270034,"PATRICK, GLORIA","Keeseville Avenue, Saint Albans, Queens County...","1 - small home, less than 4 families",19.1570%,450000.0,3761.0,19634.0,3459.0,3459.0,NaN,NaN,40.698718,-73.766175
1006440063,"FAIRFAX & SAMMONS PROPERTIES, LLC","67, Gansevoort Street, Chelsea, Manhattan, New...",4 - commercial property,10.6840%,4157000.0,157150.0,1470890.0,157150.0,157150.0,NaN,NaN,40.739595,-74.007467


In [71]:
output = []
import json
for i, row in tax_bills_bbls_sample.iterrows():
    feature = {
        "type": "Feature", 
        "geometry": {"type": "Point", "coordinates": [row['longitude'], row['latitude']]},
        "properties": {}
    }
    for key, value in row.items():
        if not pd.isnull(value):
            feature['properties'][key] = value
    output.append(feature)

with open('SampleFeatures.')
print(json.dumps(output, indent=2))

[
  {
    "geometry": {
      "type": "Point",
      "coordinates": [
        -73.9202263,
        40.7628959
      ]
    },
    "properties": {
      "emv": 818000.0,
      "ownername": "RABOS, CONSTANTINE",
      "bav": 98946.0,
      "tba": 12720.0,
      "taxclass": "2a - 4-6 unit residential building",
      "tbea": 12720.0,
      "propertytax": 12720.0,
      "address": "35th Street, Astoria, Queens County, NYC, New York, 11101, United States of America",
      "taxrate": "12.8550%",
      "longitude": -73.9202263,
      "latitude": 40.7628959
    },
    "type": "Feature"
  },
  {
    "geometry": {
      "type": "Point",
      "coordinates": [
        -73.7509159,
        40.659682
      ]
    },
    "properties": {
      "emv": 494000.0,
      "ownername": "WILSON, MARJORIE",
      "bav": 26203.0,
      "tba": 4718.0,
      "taxclass": " 1 - small home, less than 4 families",
      "tbea": 5020.0,
      "propertytax": 4718.0,
      "address": "230th Street, Laurelton, Queens Cou